# Auto Insurance Analysis

## Logistic Regression Modeling

## Project Goal

## Summary of Data

### Data Import

In [ ]:
# %run ../python_files/auto_insurance_eda

# %load ../python_files/auto_insurance_eda

## Modeling

## Results and Conclusions